<a href="https://colab.research.google.com/github/AnalyticsKnight/yemoonsaBigdata/blob/dev/src/P2_C4_02_Penguin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 4. A to Z 빅데이터 분석 실습  

## 2.실습 A to Z – Penguin

### Penguin Dataset
- https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv

In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata/

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 485 (delta 66), reused 49 (delta 44), pack-reused 363
Receiving objects: 100% (485/485), 8.89 MiB | 20.31 MiB/s, done.
Resolving deltas: 100% (251/251), done.


### (2) 사전준비

#### 422-01

In [2]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_X_test.csv")
X_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_X_train.csv")
y_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_y_train.csv")

# 사용자 코딩

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)


#### 422-02

In [3]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

#### 422-03

In [72]:
X_test = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_X_test.csv")
X_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_X_train.csv")
y_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_y_train.csv")

### (3) 데이터 전처리

#### 423-01

In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            240 non-null    object 
 1   island             240 non-null    object 
 2   sex                232 non-null    object 
 3   bill_length_mm     238 non-null    float64
 4   bill_depth_mm      238 non-null    float64
 5   flipper_length_mm  238 non-null    float64
dtypes: float64(3), object(3)
memory usage: 11.4+ KB


#### 423-02

In [73]:
train=pd.concat([X_train,y_train],axis=1)
train=train.dropna()
train.reset_index(drop=True,inplace=True)


In [74]:
#X_train=train.iloc[:,-1]
X_train=train[['species','island','sex','bill_length_mm','bill_depth_mm','flipper_length_mm']]
y_train=train[['body_mass_g']]

COL_DEL=[]
COL_NUM=['bill_length_mm','bill_depth_mm','flipper_length_mm']
COL_CAT=['species','island','sex']
COL_Y=['body_mass_g']

X=pd.concat([X_train,X_test])

from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
ohe.fit(X[COL_CAT])

X_train_res=ohe.transform(X_train[COL_CAT])
X_test_res=ohe.transform(X_test[COL_CAT])
#print(X_train_res)

X_train_ohe=pd.DataFrame(X_train_res.todense(),columns=ohe.get_feature_names())
X_test_ohe=pd.DataFrame(X_test_res.todense(),columns=ohe.get_feature_names())

X_train_fin=pd.concat([X_train[COL_NUM],X_train_ohe],axis=1)
X_test_fin=pd.concat([X_test[COL_NUM],X_train_ohe],axis=1)

from sklearn.model_selection import train_test_split

X_tr,X_val,y_tr,y_val=train_test_split(X_train_fin,y_train,test_size=0.3)

#### 423-07

In [70]:
#randomforest나 decisiontree는 불필요한 과정
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
scaler.fit(X_tr[COL_NUM])
X_tr[COL_NUM]=scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM]=scaler.transform(X_val[COL_NUM])
X_test_fin[COL_NUM]=scaler.transform(X_test_fin[COL_NUM])

from sklearn.linear_model import LinearRegression
modelLR=LinearRegression()
modelLR.fit(X_tr,y_tr)
y_val_pred=modelLR.predict(X_val)

from sklearn.metrics import mean_squared_error, r2_score

mse=mean_squared_error(y_val,y_val_pred)
rmse=mean_squared_error(y_val,y_val_pred,squared=False)

print('MSE : {0:.3f}, RMSE : {1:.3F}'.format(mse,rmse))


MSE : 94566.703, RMSE : 307.517


#### 423-08

In [75]:
from sklearn.ensemble import RandomForestRegressor

modelRF=RandomForestRegressor()
modelRF.fit(X_tr,y_tr)

y_val_predRF=modelRF.predict(X_val)
#print(y_val_pred)

from sklearn.metrics import mean_squared_error, r2_score

mse=mean_squared_error(y_val,y_val_predRF)
rmse=mean_squared_error(y_val,y_val_predRF,squared=False)

print('MSE : {0:.3f}, RMSE : {1:.3F}'.format(mse,rmse))

#print(y_pred)

MSE : 118360.550, RMSE : 344.036


#### 423-09

MSE : 67550.686, RMSE : 259.905
MSE : 86090.887, RMSE : 293.412


### (4) 데이터 모형 구축

#### 424-01

#### 424-02

#### 424-03

[[4006.97235585]
 [4675.45764127]
 [4011.39293723]
 [4778.56760262]
 [3951.26931058]
 [3923.75831329]
 [4312.16740799]
 [5418.73771001]
 [5494.57800681]
 [3456.91948485]
 [3353.71066595]
 [5292.98597359]
 [4745.35161677]
 [4176.02785456]
 [3993.74180882]
 [3283.99986207]
 [3490.23336638]
 [4756.27198988]
 [3395.74766003]
 [3516.59475917]
 [3459.3884403 ]
 [4280.22624606]
 [3329.73388938]
 [3201.31395557]
 [5579.9180549 ]
 [3981.21193695]
 [3422.35564933]
 [4285.47199276]
 [4796.05647374]
 [5522.81742696]
 [4191.41275282]
 [4277.6135177 ]
 [5351.15100549]
 [4672.97403204]
 [4362.00317253]
 [3411.96492938]
 [3316.26189932]
 [4743.75104548]
 [4143.16667003]
 [4111.90747333]
 [4054.83599991]
 [4688.69321606]
 [3405.48244076]
 [4228.53296971]
 [3625.22070086]
 [3868.16502482]
 [5669.70140431]
 [4093.64740545]
 [4191.5631587 ]
 [3542.77994916]
 [5676.27744396]
 [4696.59490996]
 [4183.5970683 ]
 [3265.82512601]
 [3110.76273212]
 [3484.16475931]
 [4088.76258516]
 [4907.06561511]
 [4154.7202624

#### 424-04

[3407.20314762]
x0_Chinstrap        -467.977365
x0_Adelie           -273.810227
x2_FEMALE           -221.289734
x1_Dream             -47.685996
x1_Torgersen           4.429691
x1_Biscoe             43.256305
x2_MALE              221.289734
bill_depth_mm        458.144803
bill_length_mm       483.046326
flipper_length_mm    705.636299
x0_Gentoo            741.787592
dtype: float64


### (5) 데이터 모형 평가

#### 425-01

MSE : 82895.238 , RMSE : 287.915 


#### 425-02

[[4158.63422978]
 [5336.6764941 ]
 [4127.30310466]
 [3452.93581447]
 [4698.72732909]
 [4033.69150879]
 [3390.84302275]
 [3754.78963146]
 [3530.60877832]
 [3485.17230841]
 [3404.97652529]
 [5494.04545069]
 [3602.48898598]
 [4088.64139673]
 [3908.55718464]
 [4105.2454243 ]
 [4355.50564106]
 [3469.69095619]
 [3564.29002084]
 [3395.63931977]
 [5365.54060484]
 [4207.38732537]
 [5840.30774131]
 [4677.49863419]
 [5408.3480874 ]
 [5507.81035945]
 [5515.09529346]
 [4240.07326899]
 [3423.95512328]
 [4049.41430402]
 [4191.68515597]
 [3417.94150254]
 [3540.73212287]
 [4636.82859851]
 [3385.884569  ]
 [3490.57518407]
 [3331.36461195]
 [3449.99547725]
 [3966.53664133]
 [4920.74019499]
 [3498.75694849]
 [5591.19864616]
 [4950.58450591]
 [4671.72645647]
 [5746.96883708]
 [3903.24102664]
 [4759.38099797]
 [3342.37782782]
 [5401.32762584]
 [4247.45790428]
 [4657.95903384]
 [4749.97086727]
 [4173.2177897 ]
 [3909.10704387]
 [3272.2104524 ]
 [4312.977993  ]
 [4809.19669191]
 [4624.03957045]
 [3887.8507115

#### 425-03

In [ ]:
pd.DataFrame({'body_mass_g': y_pred[:,0]}).to_csv('./yemoonsaBigdata/res/004000000.csv', index=False)